In [1]:
import pandas as pd
import numpy as np

In [2]:
class Particle:
    def __init__(self, n_dim, x_range, v_range):
        self.x = np.random.uniform(x_range[0], x_range[1], (no_dim))
        self.v = np.random.uniform(v_range[0], v_range[1], (no_dim))
        
        self.pbest = np.inf
        self.pbestpos = np.zeros((n_dim,))

In [3]:
class Swarm:
    def __init__(self, n_particle, n_dim, x_range, v_range, iw_ranve, c):
        self.p = np.array([Particle(n_dim, x_range, v_range) for i in range(n_particle)])
        self.gbest = np.inf
        self.gbestpos = np.zeros((n_dim,))
        self.x_range = x_range
        self.v_range = v_range
        self.iw_range = iw_range
        self.c0 = c[0]
        self.c1 = c[1]
        self.n_dim = n_dim
    
    def optimize(self, function, X, y, step, iter):
        for i in range(iter):
            for particle in self.p:
                fitness = function(X, y, particle.x)
                
                if fitness < particle.pbest:
                    particle.pbest = fitness
                    particle.pbestpos = particle.x.copy()
                
                if fitness < self.gbest:
                    self.gbest = fitness
                    self.gbestpos = particle.x.copy()
                
            for particle in self.p:
                iw = np.random.uniform(self.iw_range[0], self.iw_range[1], 1)[0]
                
                particle.v = (iw * particle.v + (self.c0 * np.random.uniform(0.0, 1.0, (self.n_dim,)) * (particle.pbestpos - particle.x)
                             + self.c1 * np.random.uniform(0.0, 1.0, (self.n_dim,)) * (self.gbestpos - particle.x)))
                
            if i % step == 0:
                print("iteration: ",i+1, "loss: ",  fitness)
            
        print("Global best loss: ", self.gbest)
        
    def best_sol(self):
        return self.gbestpos

In [4]:
data = pd.read_csv("Bank_Personal_Loan_Modelling.csv")

data = data.drop(["ZIP Code", "ID"], axis = 1)
X = data.drop(["Personal Loan"], axis = 1)
y = data.loc[:, "Personal Loan"]

In [5]:
input_neuron = X.shape[1]
hidden_neuron = 5
output_neuron = 2

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [7]:
def Negative_Likelihood(probs, Y):
    num_samples = len(probs)
    corect_logprobs = -np.log(probs[range(num_samples), Y])
    return np.sum(corect_logprobs) / num_samples

In [8]:
def softmax(logits):
    exps = np.exp(logits)
    return exps / np.sum(exps, axis=1, keepdims=True)

In [9]:
def Cross_Entropy(probs, Y):
    num_samples = len(probs)
    ind_loss = np.max(-1 * Y * np.log(probs + 1e-12), axis=1)
    return np.sum(ind_loss) / num_samples

In [10]:
def forward_pass(X, Y, W):

    if isinstance(W, Particle):
        W = W.x

    w1 = W[0 : input_neuron * hidden_neuron].reshape((input_neuron, hidden_neuron))
    b1 = W[
        input_neuron * hidden_neuron : (input_neuron * hidden_neuron) + hidden_neuron
    ].reshape((hidden_neuron,))
    w2 = W[
        (input_neuron * hidden_neuron)
        + hidden_neuron : (input_neuron * hidden_neuron)
        + hidden_neuron
        + (hidden_neuron * output_neuron)
    ].reshape((hidden_neuron, output_neuron))
    b2 = W[
        (input_neuron * hidden_neuron)
        + hidden_neuron
        + (hidden_neuron * output_neuron) : (input_neuron * hidden_neuron)
        + hidden_neuron
        + (hidden_neuron * output_neuron)
        + output_neuron
    ].reshape((output_neuron,))

    z1 = np.dot(X, w1) + b1
    a1 = np.tanh(z1)
    z2 = np.dot(a1, w2) + b2
    logits = z2

    probs = softmax(logits)
    return Negative_Likelihood(probs, Y)

In [11]:
def predict(X, W):

    w1 = W[0 : input_neuron * hidden_neuron].reshape((input_neuron, hidden_neuron))
    b1 = W[
        input_neuron * hidden_neuron : (input_neuron * hidden_neuron) + hidden_neuron
    ].reshape((hidden_neuron,))
    w2 = W[
        (input_neuron * hidden_neuron)
        + hidden_neuron : (input_neuron * hidden_neuron)
        + hidden_neuron
        + (hidden_neuron * output_neuron)
    ].reshape((hidden_neuron, output_neuron))
    b2 = W[
        (input_neuron * hidden_neuron)
        + hidden_neuron
        + (hidden_neuron * output_neuron) : (input_neuron * hidden_neuron)
        + hidden_neuron
        + (hidden_neuron * output_neuron)
        + output_neuron
    ].reshape((output_neuron,))

    z1 = np.dot(X, w1) + b1
    a1 = np.tanh(z1)
    z2 = np.dot(a1, w2) + b2
    logits = z2

    probs = softmax(logits)
    Y_pred = np.argmax(probs, axis=1)
    return Y_pred

In [12]:
def get_accuracy(Y, Y_pred):
    return (Y == Y_pred).mean()

In [14]:
no_solution = 100
no_dim = (
    (input_neuron * hidden_neuron)
    + hidden_neuron
    + (hidden_neuron * output_neuron)
    + output_neuron
)
w_range = (0.0, 1.0)
lr_range = (0.0, 1.0)
iw_range = (0.9, 0.9)
c = (0.5, 0.3)

s = Swarm(no_solution, no_dim, w_range, lr_range, iw_range, c)
s.optimize(forward_pass, X_train, y_train, 100, 1000)
W = s.best_sol()
y_pred = predict(X_test, W)
accuracy = get_accuracy(y_test, y_pred)
print("Accuracy: %.3f" % accuracy)

iteration:  1 loss:  0.9831641325522642
iteration:  101 loss:  0.9831641325522642
iteration:  201 loss:  0.9831641325522642
iteration:  301 loss:  0.9831641325522642
iteration:  401 loss:  0.9831641325522642
iteration:  501 loss:  0.9831641325522642
iteration:  601 loss:  0.9831641325522642
iteration:  701 loss:  0.9831641325522642
iteration:  801 loss:  0.9831641325522642
iteration:  901 loss:  0.9831641325522642
Global best loss:  0.3173972356303878
Accuracy: 0.904
